<a href="https://colab.research.google.com/github/mrbarron3/fwe458/blob/main/Homework4/Homework4_Barron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# imports
import pandas as pd
from google.colab import drive
from scipy.optimize import minimize
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [62]:
# Import Data
# mounting drive
drive.mount('/content/drive')

# specify the path to the file
filedir = "/content/drive/MyDrive/Spring '25/FWE458/"

fname = filedir + "Wcr_GPPdaily.csv"

# read in the csv file
df = pd.read_csv(fname)

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,TIMESTAMP,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240


In [63]:
names = {"TA_F": "TA", "SW_IN_F": "SW", "VPD_F": "VPD", "GPP_NT_VUT_REF": "GPP"}

# rename the columns and drop rows with missing data
df.rename(columns = names, inplace = True)
df.drop(axis = 1, labels = "TIMESTAMP", inplace = True)
df.dropna(inplace = True)
df.head()

,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240


In [64]:
# create new columns with interactions between variables
df["SW_VPD_int"] = df["SW"] * df["VPD"]
df["SW_TA_int"] = df["SW"] * df["TA"]
df["VPD_TA_int"] = df["VPD"] * df["TA"]
df.head()

,TA,SW,VPD,GPP,SW_VPD_int,SW_TA_int,VPD_TA_int
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209


In [66]:
# define the linear model equation: GPP=β0​+β1​⋅SW+β2​⋅VPD+β3​⋅TA+β4​⋅(SW×VPD)+β5​⋅(SW×TA)+β6​⋅(VPD×TA)
def linear_model(params, TA, SW, VPD, SW_VPD_int,	SW_TA_int,	VPD_TA_int):
    b1, b2, b3, b4, b5, b6, b0 = params
    GPPpred = b0 + b1*TA + b2*SW + b3*VPD + b4*SW_VPD_int + b5*SW_TA_int	+ b6*VPD_TA_int
    return GPPpred

In [67]:
# find the mean squared error between the prediction and actual values
def costfunc(params, TA, SW, VPD, SW_VPD_int,	SW_TA_int,	VPD_TA_int, GPP):

  # define a global prediction variable for later R_squared calculation
  global GPPpred
  GPPpred = linear_model(params, TA, SW, VPD, SW_VPD_int,	SW_TA_int,	VPD_TA_int)

  # mean squared error
  difference = np.mean((GPPpred-GPP)**2)
  return difference

# make each column in df into a series
GPP = df["GPP"]
SW = df["SW"]
VPD = df["VPD"]
TA = df["TA"]
SW_VPD_int = df["SW_VPD_int"]
SW_TA_int = df["SW_TA_int"]
VPD_TA_int = df["VPD_TA_int"]

guess = [1, 0, 0, 0, 0, 0, 0]

# minimize the costfunction by the MSE
mymin = minimize(costfunc, guess, args = (TA, SW, VPD, SW_VPD_int,	SW_TA_int,	VPD_TA_int, GPP))

b1_m, b2_m, b3_m, b4_m, b5_m, b6_m, b0_m = mymin.x
print(f"The optimized coefficients are: \nTA: {b1_m}, SW: {b2_m}, VPD: {b3_m}, \nSW_VPD_int: {b4_m}, SW_TA_int: {b5_m}, VPD_TA_int: {b6_m}, \nand intercept: {b0_m}")

The optimized coefficients are: 
TA: 82.19898653074627, SW: -3.2507613779825415, VPD: -648.9132054631987, 
SW_VPD_int: 1.7935727229964744, SW_TA_int: -0.4561990375863198, VPD_TA_int: 13.598589594151722, 
and intercept: -943.5566302451042


In [68]:
# define independent and dependent variables for the model
X = df[["TA",	"SW",	"VPD",	"SW_VPD_int",	"SW_TA_int",	"VPD_TA_int"]]
y = df["GPP"]

# fit the model
model = LinearRegression()
model.fit(X, y)

b1_l, b2_l, b3_l, b4_l, b5_l, b6_l = model.coef_
b0_l = model.intercept_
print(f"The model coefficients are: \nTA: {b1_l}, SW: {b2_l}, VPD: {b3_l}, \nSW_VPD_int: {b4_l}, SW_TA_int: {b5_l}, VPD_TA_int: {b6_l}, \nand intercept: {b0_l}")

The model coefficients are: 
TA: 82.16998132929865, SW: -3.249149679808485, VPD: -648.5741732974719, 
SW_VPD_int: 1.7924915107915305, SW_TA_int: -0.4560503569492962, VPD_TA_int: 13.595402450063668, 
and intercept: -944.0672018210355


In [70]:
# find the r-squared for the models
model_r_squ = model.score(X, y)
sci_r_squ = r2_score(df["GPP"], GPPpred)
print(f"The R-squared from SciPy's optimization is {sci_r_squ} and the R-squared for the Linear Regression model is {model_r_squ}\n")

# compare the coefficients from the two models
# a positive number means that the coefficient on that term was predicted to be greater using optimization
print(f"The difference between the coefficients from SciPy's optimization and sklearn's LinearRegression are:")
print(f"TA: {b1_m-b1_l}, SW: {b2_m-b2_l}, VPD: {b3_m-b3_l}, \nSW_VPD_int: {b4_m-b4_l}, SW_TA_int: {b5_m-b5_l}, VPD_TA_int: {b6_m-b6_l}, \nand intercept: {b0_m-b0_l}\n")
print(f"Overall, these two methods produce pretty similar predictions and very similar R-squared values.")

The R-squared from SciPy's optimization is 0.013582698131137105 and the R-squared for the Linear Regression model is 0.013582700933255087

The difference between the coefficients from SciPy's optimization and sklearn's LinearRegression are:
TA: 0.029005201447617424, SW: -0.0016116981740563752, VPD: -0.339032165726735, 
SW_VPD_int: 0.0010812122049439488, SW_TA_int: -0.00014868063702361267, VPD_TA_int: 0.0031871440880539836, 
and intercept: 0.5105715759312943

Overall, these two methods produce pretty similar predictions and very similar R-squared values.
